# Hearth Disease Prediciton with Automated ML

In [2]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core import Model, Environment
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

from azureml.pipeline.steps import AutoMLStep
import requests

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.49.0


### Compute

In [4]:
ws = Workspace.from_config("config.json")

In [5]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded.....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset 
### Overview

This data set dates from 1988 and consists of four databases: Cleveland, Hungary, Switzerland, and Long Beach V. It contains 76 attributes, including the predicted attribute, but all published experiments refer to using a subset of 14 of them. The "DEATH EVENT" field refers to the presence of heart disease in the patient. It is integer valued 0 = no disease and 1 = disease


Before we can use Azure AutoML to predict heart failures, we have to upload the dataset to the datastore and subsequently register it:

In [6]:
found = False
key = "heart_failure_data"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:

        datastore = ws.get_default_datastore()

        datastore.upload_files(files=["./heart_failure_clinical_records_dataset.csv"],
                                overwrite=True)

        # Create AML Dataset and register it into Workspace
        example_data = "heart_failure_clinical_records_dataset.csv"
        dataset = Dataset.Tabular.from_delimited_files(path=(datastore, "heart_failure_clinical_records_dataset.csv"))       
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key)


df = dataset.to_pandas_dataframe()
df

"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 1 files
Uploading ./heart_failure_clinical_records_dataset.csv
Uploaded ./heart_failure_clinical_records_dataset.csv, 1 files out of an estimated total of 1
Uploaded 1 files


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270,0
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271,0
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278,0
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280,0


In [13]:
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In the table above we can see the different features that we will use to predict if the patient is prone to suffer a hearth attack or not. Thereby we can see that the main descriptive metrics like the count, mean, standard deviation, etc. 

## AutoML Configuration

In the next step we specify the AutoML configuration. Thereby we set the timeout to 20 min in order to prevent the experiments from a timeout. The maximum of concurrent iterations is 5 because we only have a 4 nodes cluster of STANDARD_D2_V2 to run the experiment. In addition we set accuracy as the primary metric to select the best resulting model since this is the main metric to evaluate classification models.

In [ ]:
experiment_name = 'auto-ml'

experiment=Experiment(ws, experiment_name)

In [8]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             **automl_settings
                            )


In [9]:
# Submit the experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
auto-ml,AutoML_9787ad56-73f5-4a1b-9229-0969b481fb9c,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model




In [14]:
best_model, fitted_model = remote_run.get_output()

Package:azureml-automl-runtime, training version:1.51.0.post2, current version:1.49.0
Package:azureml-core, training version:1.51.0, current version:1.49.0
Package:azureml-dataprep, training version:4.10.8, current version:4.9.1
Package:azureml-dataprep-rslex, training version:2.17.12, current version:2.16.1
Package:azureml-dataset-runtime, training version:1.51.0, current version:1.49.0
Package:azureml-defaults, training version:1.51.0, current version:1.49.0
Package:azureml-interpret, training version:1.51.0, current version:1.49.0
Package:azureml-mlflow, training version:1.51.0, current version:1.49.0
Package:azureml-pipeline-core, training version:1.51.0, current version:1.49.0
Package:azureml-responsibleai, training version:1.51.0, current version:1.49.0
Package:azureml-telemetry, training version:1.51.0, current version:1.49.0
Package:azureml-train-automl-client, training version:1.51.0.post1, current version:1.49.0
Package:azureml-train-automl-runtime, training version:1.51.0.po

In [15]:
best_model.get_metrics()

{'average_precision_score_micro': 0.9226631275857157,
 'recall_score_micro': 0.8728735632183909,
 'average_precision_score_weighted': 0.9235745364025023,
 'weighted_accuracy': 0.8891861980659488,
 'f1_score_weighted': 0.8704232806342675,
 'norm_macro_recall': 0.6929921468691491,
 'AUC_macro': 0.9110885371091321,
 'f1_score_micro': 0.8728735632183909,
 'recall_score_weighted': 0.8728735632183909,
 'log_loss': 0.3519246230782122,
 'precision_score_macro': 0.8647366457072339,
 'accuracy': 0.8728735632183909,
 'precision_score_weighted': 0.8829831889806534,
 'balanced_accuracy': 0.8464960734345744,
 'AUC_micro': 0.9216337693222354,
 'average_precision_score_macro': 0.8997320855507376,
 'f1_score_macro': 0.846744208701178,
 'recall_score_macro': 0.8464960734345744,
 'precision_score_micro': 0.8728735632183909,
 'matthews_correlation': 0.7094385641396247,
 'AUC_weighted': 0.9110885371091321,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_9787ad56-73f5-4a1b-9229-0969b481fb9c_3

In [16]:
fitted_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, task='classification')),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=numpy.array([0, 1]), estimators=[('18', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.5, eta=0.5, gamma=0, max_depth=6, max_leaves=3, n_estimators=10, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=0.7291666666666667, reg_lambda=2.3958333333333335, subsample=0.8, tree_method='auto'))], verbose=False)), ('21', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrappe

## Model Deployment

In [17]:
# register the best model
model = best_model.register_model(model_name="best_automl_model", model_path="outputs/model.pkl")

In [18]:
inference_env = Environment.get(ws,"AzureML-AutoML" )

In [25]:
inference_config = InferenceConfig(entry_script="./scoring.py",
                                   environment=inference_env)

service_name = "automl-deployed-model"
deploy_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

deployed_service = Model.deploy(workspace=ws, name=service_name, models=[model],
                                inference_config=inference_config,
                                deployment_config=deploy_config)

deployed_service.wait_for_deployment(show_output=True)

deployed_service.scoring_uri

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-07-11 09:39:38+00:00 Creating Container Registry if not exists..
2023-07-11 09:49:45+00:00 Registering the environment.
2023-07-11 09:49:46+00:00 Submitting deployment to compute..
2023-07-11 09:49:50+00:00 Checking the status of deployment automl-deployed-model..
2023-07-11 09:51:46+00:00 Checking the status of inference endpoint automl-deployed-model.
Succeeded
ACI service creation operation finished, operation "Succeeded"


'http://49ecd7ae-8fe8-49ac-ac6b-bb4e67bfaabf.southcentralus.azurecontainer.io/score'

In [31]:
# Enable app insights 
deployed_service.update(enable_app_insights=True)
deployed_service.get_logs()

'2023-07-11T09:54:45,212523160+00:00 - rsyslog/run \n2023-07-11T09:54:45,231441797+00:00 - gunicorn/run \n2023-07-11T09:54:45,237685239+00:00 | gunicorn/run | \n2023-07-11T09:54:45,239617745+00:00 | gunicorn/run | ###############################################\n2023-07-11T09:54:45,241488248+00:00 | gunicorn/run | AzureML Container Runtime Information\n2023-07-11T09:54:45,247605383+00:00 | gunicorn/run | ###############################################\n2023-07-11T09:54:45,254688071+00:00 | gunicorn/run | \n2023-07-11T09:54:45,257827043+00:00 - nginx/run \n2023-07-11T09:54:45,264074485+00:00 | gunicorn/run | \n2023-07-11T09:54:45,274889578+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230628.v2\n2023-07-11T09:54:45,280749599+00:00 | gunicorn/run | \n2023-07-11T09:54:45,289292468+00:00 | gunicorn/run | \n2023-07-11T09:54:45,296731275+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/

## Send request to model endpoint

In [33]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data =  {
  "data": [
    {
      "age": 0.0,
      "anaemia": 0,
      "creatinine_phosphokinase": 0,
      "diabetes": 0,
      "ejection_fraction": 0,
      "high_blood_pressure": 0,
      "platelets": 0.0,
      "serum_creatinine": 0.0,
      "serum_sodium": 0,
      "sex": 0,
      "smoking": 0,
      "time": 0
    }
  ]
}

body = str.encode(json.dumps(data))

url = deployed_service.scoring_uri


headers = {'Content-Type':'application/json'}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'"{\\"result\\": [1]}"'


In [ ]:
deployed_service.delete()
compute_target.delete()